In [1]:
from keras.layers import Flatten, Conv2D, Conv2DTranspose, Dense, Input, Reshape, concatenate, Activation, Dropout
from keras import Sequential
from keras.models import Model

import tensorflow as tf
import keras
import sys; sys.path.insert(0, '..')
import utils

class MultiSplit(Model):
    def __init__(self, n_splits, latent_dim, io_shape):
        super(MultiSplit, self).__init__()
        n_filters = [32, 64]
        size = int( io_shape[0] // (n_splits**(1/2)) )  # W = H
        self.encoder = self._create_encoder(latent_dim//n_splits, (size, size,1), n_filters)
        self.input_reshaper = utils.Reshaper((n_splits, size, size, 1), (size, size,1))
        self.latent_reshaper = utils.Reshaper([latent_dim//n_splits], [latent_dim])
        self.classifier = self._create_classifier(latent_dim)
#         self.decoder = self._create_decoder(latent_dim, io_shape, n_filters)
        
    def train_step(self, data):
        x, y = data
        with tf.GradientTape() as tape:
            y_pred = self(x)
            loss = self.compiled_loss(y, y_pred)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def test_step(self, data):
        x, y = data
        y_pred = self(x, training=False)
        self.compiled_loss(y, y_pred)
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
    
    def call(self, x, training=True):
        z = self.encode(x, training=training)
#         y_pred = self.decode(z ,training=training)
        y_pred = self.classify(z ,training=training)
        return y_pred
        
    def encode(self, x, training=True):
        return self.encoder( self.input_reshaper(x) , training)
    
    def decode(self, z, training=True):
        return self.decoder( self.latent_reshaper(z) , training)
    
    def classify(self, z, training=True):
        return self.classifier( self.latent_reshaper(z) , training)
    
    def _create_encoder(self, latent_dim, input_shape, n_filters):
        return Sequential([
            Input(shape=input_shape),
            Conv2D(filters=n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2D(filters=n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Flatten(),
            Dense(units=latent_dim, activity_regularizer=keras.regularizers.l1(0.001))
        ], name='encoder')

    def _create_decoder(self, latent_dim, io_shape, n_filters):
        return Sequential([
            Input(shape=(latent_dim)),
            Dense(io_shape[0]//4 * io_shape[1]//4 * n_filters[0]),  # factor 4 due to conv.
            Reshape((io_shape[0]//4, io_shape[1]//4, n_filters[0])),
            Conv2DTranspose(filters=n_filters[1], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2DTranspose(filters=n_filters[0], kernel_size=(3,3), strides=(2,2), activation='relu', padding='same'),
            Conv2DTranspose(filters=1, kernel_size=(3,3), padding='same'),
            Activation('sigmoid', name='decoder_out')
        ])
    
    def _create_classifier(self, latent_dim):
        return Sequential([
            Input(shape=(latent_dim)),
            Dense(16, activation='relu'),
            Dense(16, activation='relu'),
            Dense(10, activation='softmax')
        ])

Pretrain with regularizer.

In [2]:
import keras
import numpy as np

n_splits = 1
io_shape = (28,28,1)
size = int(io_shape[0] // (n_splits**(1/2)))
latent_dim = 16

model = MultiSplit(n_splits, latent_dim, io_shape)

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_train = x_train.astype('float32') / 255.
x_train_split = np.array([utils.split(x, size, size) for x in x_train], dtype='float32')
y_train = keras.utils.to_categorical(y_train)

x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_test = x_test.astype('float32') / 255.
x_test_split = np.array([utils.split(x, size, size) for x in x_test], dtype='float32')
y_test = keras.utils.to_categorical(y_test)

In [3]:
model.compile(loss='mse', optimizer='adam') # COMPILE AFTER WEIGHTS LOADED

history = model.fit(x_train_split, x_train, validation_data=(x_test_split, x_test), epochs=25, batch_size=32)
model.encoder.save_weights('../weights/28_CE16_sparse_epoch25.h5')

Epoch 1/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0222 - val_loss: 0.0106
Epoch 2/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0100 - val_loss: 0.0091
Epoch 3/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0089 - val_loss: 0.0085
Epoch 4/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0083 - val_loss: 0.0081
Epoch 5/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0080 - val_loss: 0.0078
Epoch 6/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0077 - val_loss: 0.0076
Epoch 7/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0075 - val_loss: 0.0075
Epoch 8/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0074 - val_loss: 0.0074
Epoch 9/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0072 - val_loss: 0.0074
Epoch 10/25
1875/1875 [==============================] - 8s 4ms/step - loss: 0.007

load weights in another model and train classifier. (set encoder non trainable also fixes regularizer)

In [4]:
# model = MultiSplit(n_splits, latent_dim, io_shape)

# model.encoder.load_weights('../weights/28_CE16_sparse_epoch25.h5')
# for layer in model.encoder.layers:
#     layer.trainable = False
    
model.compile(loss='mse', metrics=['accuracy'], optimizer='adam') # COMPILE AFTER WEIGHTS LOADED

history = model.fit(x_train_split, y_train, validation_data=(x_test_split, y_test), epochs=25, batch_size=32)

Epoch 1/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0043 - accuracy: 0.9730 - val_loss: 0.0047 - val_accuracy: 0.9701
Epoch 2/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0042 - accuracy: 0.9729 - val_loss: 0.0046 - val_accuracy: 0.9706
Epoch 3/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0042 - accuracy: 0.9730 - val_loss: 0.0046 - val_accuracy: 0.9707
Epoch 4/25
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0042 - accuracy: 0.9734 - val_loss: 0.0047 - val_accuracy: 0.9689
Epoch 5/25
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0042 - accuracy: 0.9732 - val_loss: 0.0045 - val_accuracy: 0.9718
Epoch 6/25
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0041 - accuracy: 0.9738 - val_loss: 0.0047 - val_accuracy: 0.9694
Epoch 7/25
1875/1875 [==============================] - 3s 2ms/step - loss: 0.0041 - accuracy: 0.9743 - val_loss: 0.0048 - val_accuracy:

KeyboardInterrupt: 